In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('covid vaccine').getOrCreate()

In [3]:
df = spark.read.option('header', 'true').csv('covid_vaccine_statewise.csv', inferSchema=True)

df.printSchema()

root
 |-- Updated On: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Total Doses Administered: double (nullable = true)
 |-- Sessions: double (nullable = true)
 |--  Sites : double (nullable = true)
 |-- First Dose Administered: double (nullable = true)
 |-- Second Dose Administered: double (nullable = true)
 |-- Male (Doses Administered): double (nullable = true)
 |-- Female (Doses Administered): double (nullable = true)
 |-- Transgender (Doses Administered): double (nullable = true)
 |--  Covaxin (Doses Administered): double (nullable = true)
 |-- CoviShield (Doses Administered): double (nullable = true)
 |-- Sputnik V (Doses Administered): double (nullable = true)
 |-- AEFI: double (nullable = true)
 |-- 18-44 Years (Doses Administered): double (nullable = true)
 |-- 45-60 Years (Doses Administered): double (nullable = true)
 |-- 60+ Years (Doses Administered): double (nullable = true)
 |-- 18-44 Years(Individuals Vaccinated): double (nullable = true)
 |-- 45-60 

In [4]:
reqdf = df.select('State','Total Doses Administered','18-44 Years (Doses Administered)')
reqdf.show(5)

+-----+------------------------+--------------------------------+
|State|Total Doses Administered|18-44 Years (Doses Administered)|
+-----+------------------------+--------------------------------+
|India|                 48276.0|                            null|
|India|                 58604.0|                            null|
|India|                 99449.0|                            null|
|India|                195525.0|                            null|
|India|                251280.0|                            null|
+-----+------------------------+--------------------------------+
only showing top 5 rows



In [26]:
from pyspark.ml.feature import Imputer
imputer = Imputer(
                inputCols=['Total Doses Administered','18-44 Years (Doses Administered)'],
                outputCols=["{}_mean".format(val) for val in ['Total Doses Administered','18-44 Years (Doses Administered)']]).setStrategy('mean')
    
dfres = imputer.fit(reqdf).transform(reqdf)
dfres.filter(dfres['State']!='India').groupBy('State').max().show()
dfres.distinct().show()

+--------------------+-----------------------------+-------------------------------------+----------------------------------+------------------------------------------+
|               State|max(Total Doses Administered)|max(18-44 Years (Doses Administered))|max(Total Doses Administered_mean)|max(18-44 Years (Doses Administered)_mean)|
+--------------------+-----------------------------+-------------------------------------+----------------------------------+------------------------------------------+
|            Nagaland|                     791508.0|                             467353.0|                 9188170.544023095|                          8773958.21386604|
|           Karnataka|                  3.3280543E7|                          1.4653045E7|                       3.3280543E7|                               1.4653045E7|
|              Odisha|                  1.8154686E7|                            7378863.0|                       1.8154686E7|                          8773